<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [1]:
#!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
#!unzip archive.zip

Импортируем нужные библиотеки:

In [2]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords, wordnet

import gensim
import gensim.downloader as api

from torch.utils.data import Dataset, random_split

import torch.nn as nn
from torchsummary import summary

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import torch.nn as nn
from torch.optim import Adam

import gc

from tqdm.notebook import tqdm

import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

from sklearn.decomposition import PCA



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
from google.colab import drive
drive.mount("/content/drive/")
#data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])
data = pd.read_csv("/content/drive/MyDrive/training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Mounted at /content/drive/


Посмотрим на данные:

In [5]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
data.shape

(1600000, 6)

In [7]:
data = data.sample(frac=1).reset_index(drop=True)

In [8]:
data.head()

,emotion,id,date,flag,user,text
0,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
1,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
2,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
3,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
4,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem


Посмотрим на распределение по классам.

In [9]:
data['emotion'].value_counts()

0    800000
4    800000
Name: emotion, dtype: int64

Классы сбалансированы!


In [10]:
data = data[:800000]

In [11]:
data.shape

(800000, 6)

In [12]:
data['emotion'].value_counts()

0    400212
4    399788
Name: emotion, dtype: int64

Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [13]:
examples = data["text"].sample(10)
print("\n".join(examples))

yo! wasssup!? okay, I'm a little on the hyper side today!! But can you blame me!!? AHH...! Need some followers my Twitterbabes! 
It was awesome! That was a good Sci-Fi.  Tweet later! xx
16 more days 2 go back 2 india. never realized that the concept of 'day' is pretty relative. suddenly it is looking so far away 
@baragal I think i'm falling in love with you!!  xxx #iloveyou
@tonywong That's true...but lately, here in Michigan, the police have killed people via Taser Gun.  It just seems like excessive force 
Has won the high school system COLLEGE here I come! And now can't party due to first day of work 
Still sick. I feel like I am going to die. 
i just wanna say &quot;HELLO&quot; to all my new twitts/followers. feel free to get in contact with me, dnt be scared. 
and the meeting starts 
Tweetie now out as Mac client?? Gleeee. I live on the iPhone app, as I am indeed doing right now. Though I still have love for the 'deck 


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [14]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [15]:
dev_data.shape, test_data.shape

((640000, 6), (160000, 6))

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [16]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

i ' ve just realised i ' m mentally skipping tweets of green icons . partially as a result of # iranelection overload


In [17]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

just realised mentally skipping tweets green icons partially result iranelection overload


In [18]:
filtered_line

['just',
 'realised',
 'mentally',
 'skipping',
 'tweets',
 'green',
 'icons',
 'partially',
 'result',
 'iranelection',
 'overload']

## Работа с TfidfVectorizer и SVD

Проведем получение эмбеддингов слов из Tfidf с помощью TfidfVectorizer из sklearn. 

In [19]:
# обработаем слова, уберем неверную кодировку и склеим в одно предложение
def preproc_nltk(text):
    unwanted_char = '\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f\x7f\x80\x81\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x8b\x8c\x8d\x8e\x8f\x90\x91\x92\x93\x94\x95\x96\x97\x98\x99\x9a\x9b\x9c\x9d\x9e\x9f\xa0\xa1\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xab\xac\xad\xae\xaf\xb0\xb1\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xbb\xbc\xbd\xbe\xbf\xc0\xc1\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xcb\xcc\xcd\xce\xcf\xd0\xd1\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xdb\xdc\xdd\xde\xdf\xe0\xe1\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xeb\xec\xed\xee\xef\xf0\xf1\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xfb\xfc\xfd\xfe\xff'
    text = "".join([(" " if n in unwanted_char else n) for n in text if n not in unwanted_char])
    return ' '.join([w for w in tokenizer.tokenize(text.lower()) if all(c not in string.punctuation for c in w) and len(w) > 3])

In [20]:
vectorizer = TfidfVectorizer( preprocessor=preproc_nltk) #stop_words=stop_words
vectors = vectorizer.fit_transform(dev_data["text"])
vectors.shape

(640000, 337372)

In [21]:
transpose_vectors = vectors.transpose()
transpose_vectors.shape

(337372, 640000)

In [22]:
len(vectorizer.vocabulary_)

337372

In [23]:
svd = TruncatedSVD(n_components=300)
svd_matrix = svd.fit_transform(transpose_vectors)

In [24]:
svd_matrix.shape

(337372, 300)

Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [25]:
#word2vec = api.load("word2vec-google-news-300")

In [26]:

from gensim.models import KeyedVectors

filename='/content/drive/MyDrive/word2vec-google-news-300.gz'

word2vec = KeyedVectors.load_word2vec_format(filename,binary=True,unicode_errors='ignore')

In [27]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        9.1G        175M        1.2M        3.5G        4.9G
Swap:            0B          0B          0B


In [28]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


In [29]:
len(np.array(emb_line).shape)

2

Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [30]:
'''
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])
'''

'\nmean = np.mean(word2vec.vectors, 0)\nstd = np.std(word2vec.vectors, 0)\nnorm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]\nprint(sum(norm_emb_line).shape)\nprint([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])\n'

Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [31]:
class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        tokens = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        return tokens

    def get_embeddings_(self, tokens):
        result_embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in self.word2vec]
        
        if len(result_embeddings) == 0:
          result_embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
          result_embeddings = np.array(result_embeddings)
          #print(result_embeddings.shape)
          if len(result_embeddings.shape) == 1:
            result_embeddings = result_embeddings.reshape(-1, 1)            

        return result_embeddings

    def __len__(self):
        return self.data.shape[0]

In [32]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

# Average embedding
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [33]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::500]

examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [34]:
pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples['features'])#.components_

In [35]:
def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [36]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1003', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [37]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 2

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(np.array(features)), "targets": torch.LongTensor(np.array(targets))}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [38]:

def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Получим предсказания модели и посчитаем лосс
        loss = criterion(model(features), targets)

        #loss_sum += loss.item()

        # Обновим параметры модели
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")

   
def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    acc = 0
    epoch_all = 0
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получим предсказания модели и посчитаем лосс
            probs = model(features)
            loss = criterion(probs, targets) 

            # Посчитаем точность модели
            _, preds = torch.max(probs, axis=-1)
            acc = (preds == targets).sum().item()/len(features)

            mean_loss += loss.item()
            mean_acc += acc#.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}") 

    pbar.set_description(f"Test Loss: {(mean_loss / len(test_loader)):.4}, Test Acc: {(mean_acc / len(test_loader)):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [39]:
len(train), len(valid) 

(512000, 128000)

In [40]:
features = 300
classes = 2

In [41]:
model_my = nn.Sequential(
  nn.Linear(features, 256),
  nn.ReLU(),
  nn.Linear(256, 128),
  nn.ReLU(),
  nn.Linear(128, 64),
  nn.ReLU(),  
  nn.Linear(64, classes),
  nn.Tanh())
  #nn.Sigmoid())

In [42]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 3e-2
num_epochs = 1

device = "cpu"
model =  model_my
#model = model.to(device)
#model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [43]:
summary(model, (features,), batch_size=1024)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [1024, 256]          77,056
              ReLU-2                [1024, 256]               0
            Linear-3                [1024, 128]          32,896
              ReLU-4                [1024, 128]               0
            Linear-5                 [1024, 64]           8,256
              ReLU-6                 [1024, 64]               0
            Linear-7                  [1024, 2]             130
              Tanh-8                  [1024, 2]               0
Total params: 118,338
Trainable params: 118,338
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 1.17
Forward/backward pass size (MB): 7.03
Params size (MB): 0.45
Estimated Total Size (MB): 8.65
----------------------------------------------------------------


Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

**ОТВЕТ:** 
Думаю, что лосс важнее, так как во-первых, нам важно, чтоб модель выдавала максимально близкий к рельному ответ, а во-вторых, вы бы не стали просто так писать его как best metrics.
Была попытка выполнить на 20 эпохах, но точности прибавилось в тесячных, поэтому решила не гонять просто так машинку и поставить 1 эпоху. 

In [44]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/500 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/125 [00:00<?, ?it/s]

{'Test Loss': 0.5171403529644012, 'Test Acc': 0.7465625}


In [45]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/157 [00:00<?, ?it/s]

{'Test Loss': 0.5185098146936696, 'Test Acc': 0.7447750796178344}


**ВЫВОД:** 
Видим, что Accuracy = 0.7445. 

Посмотрим, улучшатся ли метрики, если смотреть контекст неизвестных слов или использовать TF-IDF.


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

ыкинуть из памяти (например загруженный word2vec):
del word2vec
после этого можно для надёжности почистить:
gc.collect()
(надо будет в начале блокнота сделать импорт import gc )
перед удалением можно сохранить на диск хотя бы командой torch.save(переменная, имя файла)
а прочитать потом:
переменная = torch.load(имя файла)
(только с word2vec такая штука не прокатит, его так сохранить не получится... Но его и не надо! - просто загрузить ещё раз из скачанного архива или через апи))

#Эмбеддинги по контексту

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста.

In [46]:
class TwitterDataset_neighbor(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.neighbor_embedding_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        tokens = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        return tokens

    def neighbor_embedding_(self, tokens):
        #f = 0
        emb_token = []
        for i in range(0, len(tokens)):
          
          w_left = np.zeros(self.word2vec.vector_size)
          w_right = np.zeros(self.word2vec.vector_size)
          
          if tokens[i] not in word2vec:
            #print(tokens[i])

            if i==0:
              for loss in range(len(tokens)-1):
                if tokens[i+1+loss] in word2vec:
                  w_right = (self.word2vec.get_vector(tokens[i+1+loss]) - self.mean) / self.std
                  break;
            
            if i == len(tokens):
              for loss in range(i-1):
                if tokens[i-1-loss] in word2vec:
                  w_left = (self.word2vec.get_vector(tokens[i-1-loss]) - self.mean) / self.std
                  break;

            else:       
            
              for loss in range(i-1):
                if tokens[i-1-loss] in word2vec:
                  w_left = (self.word2vec.get_vector(tokens[i-1-loss]) - self.mean) / self.std
                  break;
                  
              for loss in range(len(tokens)-i-1):
                if tokens[i+1+loss] in word2vec:
                  w_right = (self.word2vec.get_vector(tokens[i+1+loss]) - self.mean) / self.std
                  break;

            emb = (w_left + w_right)  
               
            emb_token.append(emb)
            

          else:
            emb = ((self.word2vec.get_vector(tokens[i]) - self.mean) / self.std)
            
            emb_token.append(emb)

        if len(emb_token) == 0:
            emb_token = np.zeros((1, self.word2vec.vector_size))    
        
        

        else:
          emb_token = np.array(emb_token)
          if len(emb_token.shape) == 1:
            emb_token = emb_token.reshape(-1, 1)

        return emb_token

    def __len__(self):
        return self.data.shape[0]

In [47]:
dev_neighbor = TwitterDataset_neighbor(dev_data, "text", "emotion", word2vec)

In [48]:
batch_size = 1024
num_workers = 2

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(np.array(features)), "targets": torch.LongTensor(np.array(targets))}


train_size = math.ceil(len(dev_neighbor) * 0.8)

train, valid = random_split(dev_neighbor, [train_size, len(dev_neighbor) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [49]:
features = 300
classes = 2

In [50]:
model_my = nn.Sequential(
  nn.Linear(features, 256),
  nn.ReLU(),
  nn.Linear(256, 128),
  nn.ReLU(),
  nn.Linear(128, 64),
  nn.ReLU(),  
  nn.Linear(64, classes),
  nn.Tanh())
  

In [51]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 3e-2
num_epochs = 1

device = "cpu"
model =  model_my
#model = model.to(device)
#model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [52]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/500 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/125 [00:00<?, ?it/s]

{'Test Loss': 0.5250398547649383, 'Test Acc': 0.739671875}


In [53]:
test_loader = DataLoader(
    TwitterDataset_neighbor(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/157 [00:00<?, ?it/s]

{'Test Loss': 0.5273037742657266, 'Test Acc': 0.7373357882165605}


** ВЫВОД:** Видим, что Accuracy стало даже чуть ниже = 0.7379. 

Посмотрим, улучшатся ли метрики, если использовать TF-IDF.

#Эмбеддинги из TF-IDF

Для каждого слова текста получим его эмбеддинг из Tfidf с помощью TfidfVectorizer из sklearn. Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf.

In [71]:

class TwitterDataset_tfidf(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, svd_matrix: svd_matrix, vectorizer: vectorizer):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.vectorizer = vectorizer

        self.svd_matrix = svd_matrix


        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        emb_final = self.get_embeddings_(tokens)
        
        return {"feature": emb_final, "target": label}

    def get_tokens_(self, text):
        tokens = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in tokens if all(c not in string.punctuation for c in w) and len(w) > 3]
        return tokens

    

    def get_embeddings_(self, tokens):
      embeddings = []
      
      for t in tokens:

        emb = np.zeros((1, word2vec.vector_size))
        if t in self.word2vec:
          emb = np.add(emb, np.array(word2vec.get_vector(t)))
        
        if t in vectorizer.vocabulary_:
          emb = np.add(emb, np.array(svd_matrix[vectorizer.vocabulary_[t]]))            
        
        if np.array_equal(emb, np.zeros((1, word2vec.vector_size))) != True:
            emb = (emb - self.mean) / self.std
            emb = emb.tolist()
            emb = np.array(emb[0])

            embeddings.append(emb)

      #if len(embeddings) > 0:
        #embeddings = np.mean(np.array(embeddings), keepdims=300, axis=0)

      if len(embeddings) == 0:
        embeddings = np.zeros((1, self.word2vec.vector_size))
      else:
        embeddings = np.array(embeddings)
        if len(embeddings.shape) == 1:
          embeddings = embeddings.reshape(-1, 1)            

      return embeddings

        
        

    def __len__(self):
        return self.data.shape[0]

In [72]:
dev_tf_new = TwitterDataset_tfidf(dev_data, "text", "emotion", word2vec, svd_matrix, vectorizer)


In [73]:
[dev_tf_new[i] for i in range(4)]

[{'feature': array([[ 1.21431761e+02, -2.46571706e+01, -4.22746464e+01, ...,
          -6.81638343e-01, -5.83620015e-01, -1.16771194e+00],
         [ 1.36445469e+00, -3.48480273e-01, -3.84371398e-01, ...,
           1.92537974e-01, -5.37382582e-01, -1.91963305e-01],
         [-1.65638549e+00,  4.57501769e-01, -2.04201410e+00, ...,
          -2.28427068e+00, -2.95956746e-02, -1.98433706e-01],
         ...,
         [ 4.50144755e-01, -9.10997004e-01, -2.06232007e-01, ...,
           1.02926113e+00,  1.75196612e+00, -1.20968534e+00],
         [ 4.56590679e-01, -1.72486656e-01, -3.06059054e-02, ...,
           2.84843555e-01, -5.98526566e-01, -3.08794823e-01],
         [ 2.32926361e+00, -9.28513762e-01, -2.67610405e+00, ...,
           6.09544251e-02, -1.14484320e+00,  1.81635906e-01]]), 'target': 0},
 {'feature': array([[ 3.04938319e+00, -3.07520290e-01, -5.45693620e-01, ...,
          -9.21167942e-02, -1.07167260e+00, -3.91821308e-01],
         [ 9.57614957e+01, -1.36903288e+01,  9.00259

In [74]:
batch_size = 1024
num_workers = 2

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(np.array(features)), "targets": torch.LongTensor(np.array(targets))}


train_size = math.ceil(len(dev_tf_new) * 0.8)

train, valid = random_split(dev_tf_new, [train_size, len(dev_tf_new) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [75]:
features = 300
classes = 2

In [76]:
model_my = nn.Sequential(
  nn.Linear(features, 256),
  nn.ReLU(),
  nn.Linear(256, 128),
  nn.ReLU(),
  nn.Linear(128, 64),
  nn.ReLU(),  
  nn.Linear(64, classes),
  nn.Tanh())
  

In [77]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 3e-2
num_epochs = 1

device = "cpu"
model =  model_my
#model = model.to(device)
#model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [78]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/500 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/125 [00:00<?, ?it/s]

{'Test Loss': 0.5338769421577454, 'Test Acc': 0.73171875}


In [79]:
test_loader = DataLoader(
    TwitterDataset_neighbor(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/157 [00:00<?, ?it/s]

{'Test Loss': 0.6062833290950508, 'Test Acc': 0.6633720640923567}


**ВЫВОД:** Видим, что Accuracy на тесте совсем низкое = 0.66 

Предполагаю, что имело бы смысл сделать лемматизацию, возможно, это бы помогло, если время не огрничивало. 

Таким образом, можно сделать вывод, что разные способы получения ембеддингов слов и далее составления из них предложений, дают примерно равные результаты. 
Однако обработка слов c TF-IDF и SVD - очень затратная операция. При прочих равных результатах предпочитительнее использовать средние эмбеддинги из предобученной модели Word2Vec.